In [8]:
# import necessary libraries
import torch
import torch.nn as nn

In [9]:
# create convolutional block
class conv_Block(nn.Module):
    def __init__(self,in_channels,out_channels,kernel=3,padding=1):
        super(conv_Block,self).__init__()
        self.conv = nn.Conv2d(in_channels,out_channels,kernel_size=kernel,stride=1,padding=padding)
        self.relu = nn.ReLU()

    def forward(self,x):
        return self.relu(self.conv(x))

In [10]:
# We can divide the net into 6 parts:
# 5 parts consist of conv_blocks(conv+relu) and in the end there is maxpolling,
# the last part is a classifier: there are fc layers and softmax in the end.
# To see more, read the paper https://arxiv.org/pdf/1409.1556.pdf
class VGG_16(nn.Module):
    def __init__(self,in_channels,num_classes):
        super().__init__()

        # 1 part
        self.seq1 = nn.Sequential(
                conv_Block(in_channels,64),
                conv_Block(64,64),
                nn.MaxPool2d(kernel_size=(2,2),stride=2))

        # 2 part
        self.seq2 = nn.Sequential(
                conv_Block(64,128),
                conv_Block(128,128),
                nn.MaxPool2d(kernel_size=(2,2),stride=2))

        # 3 part
        self.seq3 = nn.Sequential(
                conv_Block(128,256),
                conv_Block(256,256),
                conv_Block(256,256,kernel=1,padding=0),
                nn.MaxPool2d(kernel_size=(2,2),stride=2))

        # 4 part
        self.seq4 = nn.Sequential(
                conv_Block(256,512),
                conv_Block(512,512),
                conv_Block(512,512,kernel=1,padding=0),
                nn.MaxPool2d(kernel_size=(2,2),stride=2))

        # 5 part
        self.seq5 = nn.Sequential(
                conv_Block(512,512),
                conv_Block(512,512),
                conv_Block(512,512,kernel=1,padding=0),
                nn.MaxPool2d(kernel_size=(2,2),stride=2))
        
        self.classifier = nn.Sequential(
                nn.Linear(512*7*7,4096),
                nn.Linear(4096,num_classes),
                nn.Linear(num_classes,num_classes),
                nn.Softmax(dim=1))

    def forward(self,x):
        x = self.seq1(x)
        x = self.seq2(x)
        x = self.seq3(x)
        x = self.seq4(x)
        x = self.seq5(x)
        # flatten x
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x



In [11]:
# create VGG-16
def vgg_16(num_classes=1000,img_channels=3):
    return VGG_16(img_channels,num_classes)

In [12]:
# test the net architecture
def test():
    x = torch.rand(2,3,224,224)
    net = vgg_16()
    y = net(x)
    print(y.shape)
    

In [13]:
test()

torch.Size([2, 1000])
